Osnabrück University - Machine Learning (Summer Term 2024) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Lukas Niehaus

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, May 26, 2022**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Curse of Dimensionality (5 points)

For the following exercise, be detailed in your answers and provide some examples. Think about keywords like: random vectors in high dimensional space, manifolds and Bertillonage.

**a)** What are the curse of dimensionality and its implication for pattern classification?

The curse of dimensionality describes the phenomenon that in high dimensional vector spaces, two randomly drawn vectors will almost always be close to orthogonal to each other. This is a real problem in data mining, where for a higher number of features, the number of possible combinations and therefore the volume of the resulting feature space exponentionally increases.

In such a high dimensional space, data vectors from real data sets lie far away from each other (which means dense sampling becomes impossible, as there aren't enough samples close to each other). This also leads to the problem that pairs of data vectors have a high probability of having similar distances and to be close to orthogonal to each other. The result is that clustering becomes really difficult, as the vectors more or less stand on their own and distance measures cannot be applied easily.

**b)** Explain how this phenomenom could be used to one's advantage.

This is actually an advantage if you want to discriminate between a high number of individuals (see Bertillonage, where using only 11 features results in a feature space big enough to discriminate humans), but if you want to get usable information out of data, such a 'singling out' of samples is a great disadvantage.

**c)** Explain in your own words the concepts of descriptive and intrinsic dimensionality.

Intrinsic dimensionality exists in contrast to the descriptive dimensionality of data, which is defined by the numbers of parameters used to produce or represent the raw data (i.e. the number of pixels in an unprocessed image).

Additionally to this representive dimensionality, there is also a (most of the time smaller) number of independent parameters which is necessary to describe the data, always in regard to a specific problem we want to use the data on.
For example: A data set might consist of a number of portraits, all with a size of $1920\times1080$ pixels, which constitutes their descriptive dimensionality. To do some facial recognition on these portraits however, we do not need the complete descriptive dimension space (which would be way too big anyway), but only a few independent parameters (which we can get by doing PCA and looking at the eigenfaces).
This is possible because the data never fill out the entire high dimensional vector space but instead concentrate along a manifold of a much lower dimensionality.

**d)** The $n$-dimensional unit cube: A cube with edge length $d=1$ in the $n$-dimensional space $\mathbb{R}^n$. Compute the volume and the length of the diagonal for $n = 1, 2, 3, 4, 5, 10, 100, 1000$. Do the same for $d=\frac{1}{2}$. Discuss the result.

In [ ]:
from math import sqrt
d = .5  # 1.0, 0.5

for n in 1, 2, 3, 4, 5, 10, 100, 1000:
    print(f"n={n}: volume={d**n}, diagonal={sqrt(n*d**2)}")

For edge length $d=1$, the volume stays the same $1^n=1$, while the diagonal has length $\sqrt{n}$ and will grow towards $\infty$.

For $d<1$ the volume will go towards $0$ while the diagonal still grows towards $\infty$.

In high dimensions, points in the cube can be far away from each other, even if the cube is small.

**e)** The $n$-dimensional unit ball: A ball with radius $r=1$ in the $n$-dimensional space $\mathbb{R}^n$. Compute the volume for $n = 1,2,3,5,10,100,300,400$. Use the following formula to compute the volume $V$ for a given dimension $n$ and radius $R$:

$$ V_{n}(R)\sim {\frac {1}{\sqrt {n\pi }}}\left({\frac {2\pi e}{n}}\right)^{\frac {n}{2}}R^{n}$$

Refer to https://en.wikipedia.org/wiki/Volume_of_an_n-ball for more details.

Consider a $n$-dimensional unit orange consisting of the peel and the pulp: Let the thickness of the peel be $1\%$ of the radius. Compute the volume of the peel and compare to the volume of the whole orange for the same values of $n$. State the implications of your observations.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

v = lambda n, r: 1/(np.sqrt(n * np.pi)) * ((2*np.pi*np.e)/n)**(n/2)*r**n

vv = []
nn = [1,2,3,5,10,100,300,400]
for n in nn:
    print(f"n={n}, v(n,1)={v(n,1)}, v(n,.99)={v(n,.99)}, ratio pulp: {v(n,.99)/v(n,1)}")
    vv.append(v(n,.99)/v(n,1))

plt.figure()
plt.plot(nn, vv)
plt.show()

First observe, that the volume of then $n$-dimensional unit ball (and also for all other $n$-dimensional balls) tends towards $0$ for large $n$. This may be surprising at first, but if you rethink the result of assignement *d)* and consider the enclosing cube, it becomes clear that larger and larger parts of the diagonal are outside of the ball.

The second observation is, that for low dimensions, the chance that a randomly choosen point is in the pulp is high. In contrast, for higher dimensions, the probability that a random point is in the pulp goes towards $0$. High-dimensional oranges consists almost completely of peel (good that we live in a low dimensional world).

## Assignment 2: Implement and Apply PCA (8 points)

In this assignment you will implement PCA from the ground up and apply it to the `cars` dataset (simplified from the JSE [2004 New Car and Truck Data](http://jse.amstat.org/jse_data_archive.htm)). This dataset consists of measurements taken on 97 different cars. The eleven features measured are: Suggested retail price (USD), Price to dealer (USD), Engine size (liters), Number of engine cylinders, Engine horsepower, City gas mileage, Highway gas mileage, Weight (pounds), Wheelbase (inches), Length (inches) and Width (inches).

We would like to visualize these high dimensional features to get a feeling for how the cars relate to each other so we need to find a subspace of dimension two or three into which we can project the data.

In [ ]:
import numpy as np

# TODO: Load the cars dataset in cars.csv .
### BEGIN SOLUTION
cars = np.loadtxt('cars.csv', delimiter=',')
### END SOLUTION

assert cars.shape == (97, 11), "Shape is not (97, 11), was {}".format(cars.shape)

Excecute the following code which will create a scatter plot matrix (it might take some time to execute). This should give you an idea about trends and correlations in the dataset.

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set()
cols = ['Suggested retail price (USD)', 'Price to dealer (USD)',
          'Engine size (liters)', 'Number of engine cylinders',
          'Engine horsepower', 'City gas mileage' ,
          'Highway gas mileage', 'Weight (pounds)',
          'Wheelbase (inches)', 'Length (inches)', 'Width (inches)']

df = pd.DataFrame(cars, columns=cols)
sns.pairplot(df)

Looking at the scatter plot matrix above:
1. Why does the forth column/row look so different compared to the others?
2. If you would have to pick only two attributes to describe the entire dataset: Which two attributes would be the worst two pick? Why?
3. If you would have to pick only two attributes to describe the entire dataset: Which two attributes would be a better pick? Why?
4. Guess how many principle components are needed to explain 75% of the variance in the dataset if you would apply PCA to the dataset. Why?

1. The forth attribute "Number of engine cylinders" can take only positive integer values. Else something is very wrong with the car.
2. A combination of attribute one and two would be the worst pick, since they strongly correlate. I.e. describing the dataset with attribute 1 and 2 does not contain more information than just describing it with attribute 1 or 2.
3. This question is more difficult to anser. A combination of the last attribute "Width" with another attribute stretches the datapoints over a large area of the space, spanned by the two attributes, but exhibits no real structure. A combination of attribute 1 "Suggested retail price" with attribute 6 "City gas milleage" exhibits a curve like structure but uses less of the space. $\rightarrow$ Use PCA to find new optimal "attributes", i.e. principle components to describe the dataset.
4. If all scatter plots would be a straight line and thus all attributes are linerarly correlated, we could explain 100% of the variance with one principle component. Here this is not the case. But looking for example at the first attribute "Suggested retail price", we see a weak linear correlation to all other attributes. This holds true for all attribute combinations. Thus, two principle components are possible enough to explain 75% of the variance.

PCA finds a subspace that maximizes the variance by determining the eigenvectors of the covariance matrix. So we need to calculate the covariance matrix and afterwards the eigenvalues. When the data is normalized the covariance is calculated as

$$C = \frac{1}{n-1}((\bf{X}-\bar{x})^T(\bf{X}-\bar{x})) $$

with $X$ being an $n \times d$ matrix with $n$ samples and $d$ features, when $\bar{X}$ is the mean vector of features
$$\bar{X} = \frac{1}{n}\sum_{i=1}^{n}x_i.$$  

The entry $c_{i,j}$ in $C$ (a $d\times d$ matrix) tells you how much feature $i$ correlates with feature $j$.


**Note**: When the features have different scales, for achieving comparable covariance values, **first and before calculating the covariance matrix**, we need to standardize $X$ respecting its features

$${\bf{X}}_{norm} = \frac{\bf{X}}{\sigma}$$

when $\sigma$ is the standard deviation vector of features.

In [ ]:
# TODO: Compute the covariance matrix and store it into covar
# YOUR CODE HERE
### BEGIN SOLUTION
# First, we standardize the matrix
std_vec = np.std(cars, axis=0)
cars_norm = cars / std_vec

# Then we calculate the covariance matrix
mean_vec = np.mean(cars_norm, axis=0)
covar = (cars_norm - mean_vec).T @ (cars_norm - mean_vec) / (cars_norm.shape[0] - 1)

# Alternatively one could use:
# np.cov(X, rowvar=False)
# Note: using the np.cov method you don't need to normalize your data
### END SOLUTION

assert covar.shape == (11, 11)

# TODO: Compute the eigenvalues and eigenvectors and store them into eigenval and eigenvec
#       (Figure out a function to do this for you)
# YOUR CODE HERE
### BEGIN SOLUTION
eigenval, eigenvec = np.linalg.eigh(covar)

# Alternatively, np.linalg.eig solves the eigenvector problem for general
# matrices, while eigh takes advantage of the covariance matrix symmetry and
# uses a faster algorithm. Plus, it returns eigenvalues sorted in ascending
# order.
# eigenval, eigenvec = np.linalg.eig(covar)

### END SOLUTION

assert eigenval.shape == (11,)
assert eigenvec.shape == (11, 11)
for ev in eigenvec: np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

Plot the spectrum of the eigenvalues and make sure that they are sorted by their magnitude (in descending order).

**Note:** Sorting should be done respecting eigenvalues vector, but the order of eigenvectors should also be updated such that the corresponding pairs of eigenvalue-eigenvector be accessible with the same index.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### BEGIN SOLUTION
# Sort eigenvectors (and -values) by descending order of eigenvalues.
sort = np.argsort(-eigenval)
eigenval = eigenval[sort]
eigenvec = eigenvec[:,sort]

# To get an idea of the eigenvalues we plot them.
figure = plt.title('Eigenvalues Comparison')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalues')
plt.bar(np.arange(len(eigenval)), eigenval);
### END SOLUTION

assert not isinstance(eigenval, np.matrix)
assert not isinstance(eigenvec, np.matrix)

How many principal components should you include based on the spectrum plot?

One method to decide about the number of components is the "explained variance." The amount of data variance captured by each principal components is the magnitude of its corresponding eigenvalue. Therefore, in the explained variance method, we calculate the proportion of each the eigenvalue respective to the total sum of the eigenvalues. That gives us the percentage of data variance explained by each corresponding principal component. And the cumulative sum of these percentages shows how much more of the dataset information (variance) is presentable with taking one more component. Knowing that on one hand, and the computational cost and the difficulty of visualization of one more dimension gives a clue for the decision.

Execute the cells below and decide about an efficient number of principal components.

In [ ]:
var_exp = [val*100/sum(eigenval) for val in eigenval]
cum_sum = np.cumsum(var_exp)

fig, ax = plt.subplots()
ind = range(1, len(eigenval)+1)
plt.bar(ind, var_exp, label='Explained Variance')
plt.step(ind, cum_sum, 'r:',
         where='mid', label='Cumulative Sum')
plt.ylabel('Explained Variance (%)')
plt.xlabel('Principal Components')
ax.set_xticks(ind)
plt.legend(loc='center right');

As you can see, with the first eigenvalue/component we can describe about 70% of the variance in our dataset; taking the first two will increase it to more than 80%. For the sake of a convenient visualization we will go with the first two components, because the increase of the variation is not that much after the second component.

Now you should have a matrix full of eigenvectors. We can now do two things: project the data down onto the two dimensional subspace to visualize it and we can also plot the two first principle component vectors as eleven two dimensional points to get a feeling for how the features are projected into the subspace. Execute the two cells below and describe what you see. Is PCA a good method for this problem? Was it justifiable that we only considered the first two principle components? What kinds of cars are in the four quadrants of the second plot? (**put your answer in the cell below of this code cell**)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Project the data down into the two dimensional subspace
proj = cars_norm @ eigenvec[:,:2]


# Plot projected data
plt.title('Data Projected Onto First two Principal Components')
plt.xticks([])
plt.yticks([])
plt.xlabel('PC1 ({}%)'.format(round(var_exp[0])))
plt.ylabel('PC2 ({}%)'.format(round(var_exp[1])))
plt.scatter(proj[:,0], proj[:,1], alpha=.4)

# Plot the PC Vectors
# Project them and scale them by the standard deviation.
eigenvec_p = eigenvec.T @ eigenvec * np.sqrt(eigenval)
origin = np.mean(proj, axis=0)

plt.quiver(*origin, eigenvec_p[0,0], eigenvec_p[0,1], angles='xy', scale_units='xy', scale=1)
plt.quiver(*origin, eigenvec_p[1,0], eigenvec_p[1,1], angles='xy', scale_units='xy', scale=1)


plt.text(-12,-17.5, 'PC1')
plt.text(-18,-15, 'PC2');

In [ ]:
# Set the plot
plt.figure(figsize=(10,8))

plt.title('Eigenvectors Plot')
plt.axhline(0, color='green', linestyle=':')
plt.axvline(0, color='green', linestyle=':')
plt.xticks([])
plt.yticks([])
plt.xlabel('PC1 ({}%)'.format(round(var_exp[0])))
plt.ylabel('PC2 ({}%)'.format(round(var_exp[1])))


# plot centered projected data
proj = proj - np.mean(proj, axis=0)
plt.scatter(proj[:,0], proj[:,1], alpha=.4)

# scale eigenvectors
eigenvec_s = eigenvec * np.sqrt(eigenval)

# Plot the eigenvector and add the labels
for idx, eivec in enumerate(eigenvec_s[:,:2]):
    plt.arrow(0, 0, eivec[0]*5, eivec[1]*5, alpha=.8, 
              color=plt.get_cmap('Set3')(idx), 
              width=0.03, head_width=.2, label=cols[idx])   

plt.legend()
plt.show()

#### PCA is good
The first plot shows the complete dataset projected down onto the two first principle components. Only few points overlap and the points are generally spread out well in the subspace. There is not much trend in the plot which is what we desired, i.e. the axes are not redundant. No clusters can be recognized.

#### The first two PCs are good
It is admissible to pick two dimensions, although only the first eigenvector has a very high magnitude in comparison.

The 1D plot, which might be better if we take the eigenvalues into account, yields little information in the data on a visual level. However, it gives a better idea of how the original features will be distributed in the space.

The 3D plot is already very hard to grasp by just taking a look at it. So 2D seems to be a good choice.

In general there are several different strategies to decide the number of dimensions onto which you want to project your data. Here is a short overview over a couple of more common choices:

- Visualization: Choose the number of dimensions which is useful to visualize the data in a meaningful way. This
  choice depends a lot on your problem definition. For printing 2D is usually a good choice - but maybe your data
  is just very nice for 1D already. Or maybe you are using a glyph plot (see sheet 06) which can represent high
  dimensional data.
- Classification results: In the Eigenfaces assignment below we figured out that the number of principal
  components (and thus the number of dimensions) can have a crucial impact on classification rates. It is thus
  an option to fine tune the number of dimensions for a good classification result on some training set.

#### Interpretation of the plot is very subjective
Let's first take a look at the second plot. Each of the eleven points denotes one of the original base vectors. And the vectors are projections of the original axes.

- **Top right**: Cars with high gas mileages. This might be limousines.
- **Bottom right**: Cars with low prices and low power, but average sizes and higher gas mileages. This might also be
  limousines, but smaller ones.
- **Bottom left**: Cars with big measurements and average pricing. This might be family cars.
- **Top left**: Cars with considerably high power and prices which are still light and small.
  This might be sports cars.

Note that this interpretation is just describing the general trend. Due to the nature of linear combinations, it is easily possible to come up with a car which has some exceptional values which lead to cancellation of others.

Note also that depending on the method used to calculate the eigenvectors, your axes and thus your interpretation might slightly differ.

## Assignment 3: PCA (7 points)

In this exercise we investigate the statement from the lecture that PCA finds the subspace that captures most of the data variance. To be more precise, we show that the orthonormal projection onto an $m$-dimensional subspace that maximizes the variance of the projected data is defined by the principal components, i.e. by the $m$ eigenvectors of the covariance matrix $C$ corresponding to the $m$ largest eigenvalues. The proof consists of two steps.

**a) First step:** Consider a one dimensional subspace: Determine a (unit) vector $\vec{p}$, such that the variance of the data, when projected onto the subspace determined by that vector, is maximal.

The covariance matrix $C$ allows to compute the variance of the projected data as $\vec{p}^{T}C\vec{p}$. We want to maximize this expression. To avoid $\|\vec{p}\|\to\infty$ we will only consider unit vectors, i.e. we constrain $\vec{p}$ to be normalized: $\vec{p}^T\vec{p}=1$. Maximize the expression with this constraint (which can be done using a Lagrangian multiplier). Conclude that a suitable $\vec{p}$ has to be an eigenvector of $C$ and describe which of the eigenvectors is optimal.

**Solution:**
We want to maximize the expression
$$\vec{p}^T C\vec{p} + \lambda(1-\vec{p}^T\vec{p})$$
with respect to $\vec{p}$, i.e. we have to find solutions for
$$\frac{\partial}{\partial\vec{p}}\left[ \vec{p}^T C\vec{p} + \lambda(1-\vec{p}^T\vec{p})\right] = 0$$
This leads to the equation
$$C\vec{p} = \lambda\vec{p}$$
In other words: For a vector $\vec{p}$ to maximize our expression, it has to be an eigenvector $C$ and $\lambda$ has to be the corresponding eigenvalue.
By left multiplying with $\vec{p}^T$ and using the fact that $\vec{p}^T\vec{p}=1$, we gain
$$\vec{p}^TC\vec{p}=\lambda$$
i.e. the projected variance will correspond to the eigenvalue $\lambda$ and hence is maximized when choosing the largest eigenvalue.

**b) Second step:** Now proof the statement for the general case of an $m$-dimensional projection space.

Use an inductive argument: assume the statement has been shown for the $(m-1)$-dimensional projection space, spanned by the $m-1$ (orthonormal) eigenvectors $\vec{p}_1,\ldots,\vec{p}_{m-1}$ corresponding to the $(m-1)$ largest eigenvalues $\lambda_1,\ldots,\lambda_{m-1}$. Now find a (unit) vector $\vec{p}_m$, orthogonal to the existing vectors $\vec{p}_1,\ldots,\vec{p}_{m-1}$, that maximizes the projected variance $\vec{p}_m^TC\vec{p}_m$. Proceed similar to case (a), but with additional Lagrangian multipliers to enforce the orthogonality constraint. Show that the new vector $\vec{p}_m$ is an eigenvector of $C$. Finally show that the variance is maximized for the eigenvector corresponding to the $m$-th largest eigenvalue $\lambda_m$.

**Solution:** Assume that the result holds for projection spaces of dimensionality $m-1$. We will now show that it then also holds for dimensionality $m$: we consider a subspace spanned by the $m-1$ (orthonormal) eigenvectors $\vec{p}_1,\ldots,\vec{p}_{m-1}$ corresponding to the $(m-1)$ largest eigenvalues $\lambda_1,\ldots,\lambda_{m-1}$, and a new vector $\vec{p}_{m}$ whos properties we will now examine. First, this vector should be linearly independent from $\vec{p}_1,\ldots,\vec{p}_{m-1}$, as it should define the new $m$-th dimension. The property can be enforced by the (stronger) requirement that $\vec{p}_{m}$ should be orthogonal to $\vec{p}_1,\ldots,\vec{p}_{m-1}$, i.e. 
$$\vec{p}_m^T\vec{p}_{i}=0 \text{ for } i=1,\ldots,m-1,$$
which can be expressed using Lagrange multipliers $\eta_1,\ldots,\eta_{m-1}$. As argued in part (a), the variance in direction $\vec{p}_m$ is given by
$$\vec{p}_{m}^TC\vec{p}_{m}.$$
We want to maximize this value, again with the additional constraint that $\vec{p}_{m}$ is normalized, i.e.
$$\vec{p}_{m}^T\vec{p}_m=1,$$
which will be expressed by an additional Lagrange multiplier $\lambda_M$. So in total we want to maximize the function
$$\vec{p}_{m}^TC\vec{p}_{m} + \sum_{i=1}^{m-1}\eta_i\vec{p}_m^T\vec{p}_{i} + \lambda_m(1-\vec{p}_{m}^T\vec{p}_{m})$$
with respect to $\vec{p}_m$, i.e. we have to find solutions for
\begin{align}
  0
  & = \frac{\partial}{\partial\vec{p}_m}\left[\vec{p}_{m}^TC\vec{p}_{m} 
  + \sum_{i=1}^{m-1}\eta_i\vec{p}_m^T\vec{p}_{i}
  + \lambda_m(1-\vec{p}_{m}^T\vec{p}_m)\right] \\
  & = 2C\vec{p}_m + \sum_{i=1}^{m-1}\eta_i\vec{p}_{i} - 2\lambda_m\vec{p}_{m}
\end{align}
Multiplying this equation with $\vec{p}_{j}^T$ from the left yields (due to the orthogonality constraint)
\begin{align}
  0 = \vec{p}_{j}^T 0
  & = \vec{p}_{j}^T 2C\vec{p}_m +
  \vec{p}_{j}^T \sum_{i=1}^{m-1}\eta_i\vec{p}_{i} -
  \vec{p}_{j}^T 2\lambda_m\vec{p}_{m} \\
  &= 0 + \eta_j\vec{p}_{j}^T \vec{p}_{j}- 0 \\
  & = \eta_j
\end{align}
for $j=1,\ldots,m-1$. So the problem simplifies to
$$0 = 2C\vec{p}_m - 2\lambda_m\vec{p}_{m}$$
from which we see that a critical point of the Lagrange equation has to fulfill
$$C\vec{p}_m =\lambda_m\vec{p}_{m}$$
which just means it has to be an eigenvector of the matrix $C$ with eigenvalue $\lambda_M$. There may be multiple eigenvectors for $C$, so we have to select $\vec{p}_m$ in a way that it maximizes the variance in direction $\vec{p}_m$, i.e. the value
$$\vec{p}_{m}^TC\vec{p}_{m} = \vec{p}_{m}^T\lambda_M\vec{p}_{m} = \lambda_M.$$
This just means that we have to choose $\vec{p}_m$ to be the eigenvector with the largest eigenvalue (amongst those not previously selected). This completes the inductive step.